In [12]:
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
import os
import cv2
import math
import numpy as np
import copy
import itertools
import csv
from collections import Counter
from collections import deque
from utils import CvFpsCalc
from matplotlib import pyplot as plt

In [13]:
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480


def resize_and_show(image):
  h, w = image.shape[:2]
  if h < w:
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  else:
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))
  return img

def load_images_from_folder(folder):
    images = []
    
    for filename in os.listdir(folder):
        #print(filename)
        os.chdir(folder)
        img = cv2.imread(filename)
        #print(img)
        if img is not None:
            img=resize_and_show(img)
            images.append(img)
    return images

In [14]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

In [15]:
def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))
    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

In [16]:
def logging_csv(number, mode, landmark_list, point_history_list):
# if mode == 0:
#     pass
# if mode == 1 and (0 <= number <= 9):
    csv_path = 'model/keypoint_classifier/keypoint_key_letters_right_moving.csv'
    #csv_path = 'model/keypoint_classifier/keypoint_withoutQ&X.csv'


    with open(csv_path, 'a', newline="") as f:
        writer = csv.writer(f)
        writer.writerow([number, *landmark_list])

    return

In [ ]:
#Land mark collection
root_dir="C:\\Users\\User\\Desktop\\Sign_language\\PolishAlphabet_SignLanguage_Detection\\key_letters_aug"
letters=os.listdir(root_dir)
letters.sort()
label=-1
for letter in letters:
    label=label+1
    input_dir =os.path.join(root_dir,letter)
    images=load_images_from_folder(input_dir)
    os.chdir("C:\\Users\\User\\Desktop\\Sign_language\\PolishAlphabet_SignLanguage_Detection")
    first = True
    for image in images:
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.7) as hands:
        
            results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

            tmp = "No"
            index = 0

            ###############################
            # RIGHT HAND DETECTION
            if (results.multi_hand_landmarks is not None):
                for idx,hand in enumerate(results.multi_handedness):
                    #print(hand.classification[0].label)
                    if(str(hand.classification[0].label)=="Right"):
                        # print(hand.classification[0].label)
                        # print(idx)
                        index = idx
                        break
            ###############################
            # print(index)

            if (results.multi_hand_landmarks is not None):
                # Landmark calculation
                landmark_list = calc_landmark_list(image, results.multi_hand_landmarks[index])


                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                        landmark_list)

                # print(pre_processed_landmark_list)
                # Write to the dataset file
                logging_csv(label, 1, pre_processed_landmark_list,[])                

In [17]:
# relative_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
relative_list = []
def relative_change(x_1,y_1,x_2,y_2,distance):
    if(x_2==999 or y_2==999):
        return 0,0
    new_x = (x_1-x_2)/distance
    new_y = (y_1-y_2)/distance
    
    return new_x,new_y
    
    

In [42]:
thisdict = { 
    1: [7],
    2: [3,7],
    3: [3,5,7],
    4: [2,4,5,7],
    5: [2,3,4,5,7],
    6: [2,3,4,5,6,7],
    7: [1,2,3,4,5,6,7]
}

In [18]:
# ===================================================================================================
#Land mark collection
root_dir="C:\\Users\\User\\Desktop\\moving_frames\\F\\1"
distance = 1

beginning_coordinates= [0,0]
coordinates_with_movement = []

label=-1

relative_landmark_list = []
tmp=0
images=load_images_from_folder(root_dir)
for image in images:
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
        
        results = hands.process(cv2.flip(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), 1))

        index = 0

        ###############################
        # RIGHT HAND DETECTION
        if (results.multi_hand_landmarks is not None):
            for idx,hand in enumerate(results.multi_handedness):
                #print(hand.classification[0].label)
                if(str(hand.classification[0].label)=="Right"):
                    # print(hand.classification[0].label)
                    # print(idx)
                    index = idx
                    break
        ###############################
        # print(index)
        if (results.multi_hand_landmarks is not None):
            # Landmark calculation
            landmark_list = calc_landmark_list(image, results.multi_hand_landmarks[index])
            
            # print(distance)
            
            # Conversion to relative coordinates / normalized coordinates
            if(tmp==0):
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)
                relative_landmark_list = pre_processed_landmark_list

                beginning_coordinates= [landmark_list[12][0],landmark_list[12][1]]
                #d=√((x_2-x_1)²+(y_2-y_1)²)
                distance = int(math.sqrt(math.pow(landmark_list[0][0]-landmark_list[5][0],2)+math.pow(landmark_list[0][1]-landmark_list[5][1],2)))
                if(distance==0):
                    distance=1
            
            if(tmp!=0):
                relative_list.append(relative_change(beginning_coordinates[0],beginning_coordinates[1],landmark_list[12][0],landmark_list[12][1],distance))
            # print(pre_processed_landmark_list)
            # # Write to the dataset file
            # logging_csv(label, 1, pre_processed_landmark_list,[])  
    print(str(tmp))
    tmp=tmp+1 

0
1
2
3
4
5
6
7


In [19]:
print(relative_list)

[(0.15384615384615385, 0.5), (1.4615384615384615, 0.8461538461538461), (1.6923076923076923, -0.3076923076923077), (1.1923076923076923, -0.19230769230769232), (1.2692307692307692, -0.19230769230769232), (1.2692307692307692, -0.19230769230769232), (0.038461538461538464, 0.038461538461538464)]


In [57]:
end_coordinates = relative_list[-1]
count=0
# for i in relative_list:
#     relative_x[count]=i[0]
#     relative_y[count]=i[1]
#     count = count +1

In [58]:
print(relative_x)
print(relative_y)

[0.15384615384615385, 1.4615384615384615, 1.6923076923076923, 1.1923076923076923, 1.2692307692307692, 1.2692307692307692, 0.038461538461538464]
[0.5, 0.8461538461538461, -0.3076923076923077, -0.19230769230769232, -0.19230769230769232, -0.19230769230769232, 0.038461538461538464]


In [20]:
# filling empty frames linearly
def fill_coordinates(relative_list):
    list_x=[0,0,0,0,0,0,0]
    list_y=[0,0,0,0,0,0,0]
    if(len(relative_list)==0):
        return list_x
    if(len(relative_list)==1):
        list_x[6]=relative_list[0][0]
        list_y[6]=relative_list[0][1]

        # filling 0 values

        list_x[5]= list_x[6] * 6 / 7
        list_y[5]= list_y[6] * 6 / 7

        list_x[4]=list_x[6] * 5 / 7
        list_y[4]=list_y[6] * 5 / 7

        list_x[3]=list_x[6] * 4 / 7
        list_y[3]=list_y[6] * 4 / 7

        list_x[2]=list_x[6] * 3 / 7
        list_y[2]=list_y[6] * 3 / 7

        list_x[1]=list_x[6] * 2 / 7
        list_y[1]=list_y[6] * 2 / 7

        list_x[0]=list_x[6] * 1 / 7
        list_y[0]=list_y[6] * 1 / 7
    
    if(len(relative_list)==2):
        list_x[6]=relative_list[1][0]
        list_y[6]=relative_list[1][1]

        list_x[3]=relative_list[0][0]
        list_y[3]=relative_list[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] - list_x[3]) + (list_x[6] - list_x[3]) * 2 / 3 
        list_y[5]=(list_y[6] - list_y[3]) + (list_y[6] - list_y[3]) * 2 / 3 

        list_x[4]=(list_x[6] - list_x[3]) + (list_x[6] - list_x[3]) * 1 / 3
        list_y[4]=(list_y[6] - list_y[3]) + (list_y[6] - list_y[3]) * 1 / 3 

        list_x[2]=(list_x[3]) * 3 / 4
        list_y[2]=(list_y[3]) * 3 / 4

        list_x[1]=(list_x[3]) * 2 / 4
        list_y[1]=(list_y[3]) * 2 / 4

        list_x[0]=(list_x[3]) * 1 / 4
        list_y[0]=(list_y[3]) * 1 / 4
    
    if(len(relative_list)==3):
        list_x[6]=relative_list[2][0]
        list_y[6]=relative_list[2][1]

        list_x[4]=relative_list[1][0]
        list_y[4]=relative_list[1][1]

        list_x[2]=relative_list[0][0]
        list_y[2]=relative_list[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[3]=(list_x[4] + list_x[2]) / 2
        list_y[3]=(list_y[4] + list_y[2]) / 2

        list_x[1]=(list_x[2]) * 2 / 3
        list_y[1]=(list_y[2]) * 2 / 3

        list_x[0]=(list_x[2]) * 1 / 3
        list_y[0]=(list_y[2]) * 1 / 3
    
    if(len(relative_list)==4):
        list_x[6]=relative_list[3][0]
        list_y[6]=relative_list[3][1]

        list_x[4]=relative_list[2][0]
        list_y[4]=relative_list[2][1]

        list_x[3]=relative_list[1][0]
        list_y[3]=relative_list[1][1]

        list_x[1]=relative_list[0][0]
        list_y[1]=relative_list[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2

        list_x[0]=(list_x[1]) / 2
        list_y[0]=(list_y[1]) / 2

    if(len(relative_list)==5):
        list_x[6]=relative_list[4][0]
        list_y[6]=relative_list[4][1]

        list_x[4]=relative_list[3][0]
        list_y[4]=relative_list[3][1]

        list_x[3]=relative_list[2][0]
        list_y[3]=relative_list[2][1]

        list_x[1]=relative_list[1][0]
        list_y[1]=relative_list[1][1]

        list_x[0]=relative_list[0][0]
        list_y[0]=relative_list[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list)==6):
        list_x[6]=relative_list[5][0]
        list_y[6]=relative_list[5][1]

        list_x[5]=relative_list[4][0]
        list_y[5]=relative_list[4][1]

        list_x[4]=relative_list[3][0]
        list_y[4]=relative_list[3][1]

        list_x[3]=relative_list[2][0]
        list_y[3]=relative_list[2][1]

        list_x[1]=relative_list[1][0]
        list_y[1]=relative_list[1][1]

        list_x[0]=relative_list[0][0]
        list_y[0]=relative_list[0][1]

        # filling 0 values

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list)==7):
        list_x[6]=relative_list[6][0]
        list_y[6]=relative_list[6][1]
        
        list_x[5]=relative_list[5][0]
        list_y[5]=relative_list[5][1]

        list_x[4]=relative_list[4][0]
        list_y[4]=relative_list[4][1]

        list_x[3]=relative_list[3][0]
        list_y[3]=relative_list[3][1]

        list_x[2]=relative_list[2][0]
        list_y[2]=relative_list[2][1]

        list_x[1]=relative_list[1][0]
        list_y[1]=relative_list[1][1]

        list_x[0]=relative_list[0][0]
        list_y[0]=relative_list[0][1]

    return list_x,list_y

In [31]:
def linear_approx_x(x_1,x_2):
    points_x = []
    if(x_1>=0 and x_2>=0):
        if(x_1 > x_2):
            points_x.append(x_2 + ((x_1 - x_2) * 2 /3))
            points_x.append(x_2 + ((x_1 - x_2) * 1 /3))
            return points_x
        else:
            points_x.append(x_1 + ((x_2 - x_1) * 1 /3))
            points_x.append(x_1 + ((x_2 - x_1) * 2 /3))
            return points_x

    if(x_1>=0 and x_2<=0):
        points_x.append(x_2 + ((x_1 + abs(x_2)) * 2 /3))
        points_x.append(x_2 + ((x_1 + abs(x_2)) * 1 /3))
        return points_x

    if(x_1<=0 and x_2>=0):
        points_x.append(x_1 + ((x_2 + abs(x_1)) * 1 /3))
        points_x.append(x_1 + ((x_2 + abs(x_1)) * 2 /3))
        return points_x
    
    if(x_1<=0 and x_2<=0):
        if(x_1 > x_2):
            points_x.append(x_1 + ((x_2)-(x_1)) * 1/3)
            points_x.append(x_1 + ((x_2)-(x_1)) * 2/3)
            return points_x
        else:
            points_x.append(x_1 + ((x_2)-(x_1)) * 1/3)
            points_x.append(x_1 + ((x_2)-(x_1)) * 2/3)
            return points_x
    


In [32]:
def linear_approx_y(y_1,y_2):
    points_y = []
    if(y_1>=0 and y_2>=0):
        if(y_1 > y_2):
            points_y.append(y_2 + ((y_1 - y_2) * 2 /3))
            points_y.append(y_2 + ((y_1 - y_2) * 1 /3))
            return points_y
        else:
            points_y.append(y_1 + ((y_2 - y_1) * 1 /3))
            points_y.append(x_1 + ((y_2 - y_1) * 2 /3))
            return points_y

    if(y_1>=0 and y_2<=0):
        points_y.append(y_2 + ((y_1 + abs(y_2)) * 2 /3))
        points_y.append(y_2 + ((y_1 + abs(y_2)) * 1 /3))
        return points_y

    if(y_1<=0 and y_2>=0):
        points_y.append(y_1 + ((y_2 + abs(y_1)) * 1 /3))
        points_y.append(y_1 + ((y_2 + abs(y_1)) * 2 /3))
        return points_y
    
    if(y_1<=0 and y_2<=0):
        if(y_1 > y_2):
            points_y.append(y_1 + ((y_2)-(y_1)) * 1/3)
            points_y.append(y_1 + ((y_2)-(y_1)) * 2/3)
            return points_y
        else:
            points_y.append(y_1 + ((y_2)-(y_1)) * 1/3)
            points_y.append(y_1 + ((y_2)-(y_1)) * 2/3)
            return points_y
    


In [1]:
# filling empty frames linearly
def fill_coordinates(relative_list_in_fnc):
    list_x=[0,0,0,0,0,0,0]
    list_y=[0,0,0,0,0,0,0]
    if(len(relative_list_in_fnc)==0):
        return list_x
    if(len(relative_list_in_fnc)==1):
        list_x[6]=relative_list_in_fnc[0][0]
        list_y[6]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]= list_x[6] * 6 / 7
        list_y[5]= list_y[6] * 6 / 7

        list_x[4]=list_x[6] * 5 / 7
        list_y[4]=list_y[6] * 5 / 7

        list_x[3]=list_x[6] * 4 / 7
        list_y[3]=list_y[6] * 4 / 7

        list_x[2]=list_x[6] * 3 / 7
        list_y[2]=list_y[6] * 3 / 7

        list_x[1]=list_x[6] * 2 / 7
        list_y[1]=list_y[6] * 2 / 7

        list_x[0]=list_x[6] * 1 / 7
        list_y[0]=list_y[6] * 1 / 7
    
    if(len(relative_list_in_fnc)==2):
        list_x[6]=relative_list_in_fnc[1][0]
        list_y[6]=relative_list_in_fnc[1][1]

        list_x[3]=relative_list_in_fnc[0][0]
        list_y[3]=relative_list_in_fnc[0][1]

        # filling 0 values
        # The only case with if:

        # list_x[5]=abs((list_x[6] - list_x[3])) + (list_x[6] - list_x[3]) * 2 / 3 
        # list_y[5]=abs((list_y[6] - list_y[3])) + (list_y[6] - list_y[3]) * 2 / 3 

        # list_x[4]=abs((list_x[6] - list_x[3])) + (list_x[6] - list_x[3]) * 1 / 3
        # list_y[4]=abs((list_y[6] - list_y[3])) + (list_y[6] - list_y[3]) * 1 / 3 

        coords_x = linear_approx_x(list_x[6],list_x[3])
        coords_y = linear_approx_x(list_y[6],list_y[3])

        list_x[5]= coords_x[0]
        list_y[5]= coords_y[0]

        list_x[4]= coords_x[1]
        list_y[4]= coords_y[1]

        list_x[2]=(list_x[3]) * 3 / 4
        list_y[2]=(list_y[3]) * 3 / 4

        list_x[1]=(list_x[3]) * 2 / 4
        list_y[1]=(list_y[3]) * 2 / 4

        list_x[0]=(list_x[3]) * 1 / 4
        list_y[0]=(list_y[3]) * 1 / 4
    
    if(len(relative_list_in_fnc)==3):
        list_x[6]=relative_list_in_fnc[2][0]
        list_y[6]=relative_list_in_fnc[2][1]

        list_x[4]=relative_list_in_fnc[1][0]
        list_y[4]=relative_list_in_fnc[1][1]

        list_x[2]=relative_list_in_fnc[0][0]
        list_y[2]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[3]=(list_x[4] + list_x[2]) / 2
        list_y[3]=(list_y[4] + list_y[2]) / 2

        list_x[1]=(list_x[2]) * 2 / 3
        list_y[1]=(list_y[2]) * 2 / 3

        list_x[0]=(list_x[2]) * 1 / 3
        list_y[0]=(list_y[2]) * 1 / 3
    
    if(len(relative_list_in_fnc)==4):
        list_x[6]=relative_list_in_fnc[3][0]
        list_y[6]=relative_list_in_fnc[3][1]

        list_x[4]=relative_list_in_fnc[2][0]
        list_y[4]=relative_list_in_fnc[2][1]

        list_x[3]=relative_list_in_fnc[1][0]
        list_y[3]=relative_list_in_fnc[1][1]

        list_x[1]=relative_list_in_fnc[0][0]
        list_y[1]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2

        list_x[0]=(list_x[1]) / 2
        list_y[0]=(list_y[1]) / 2

    if(len(relative_list_in_fnc)==5):
        list_x[6]=relative_list_in_fnc[4][0]
        list_y[6]=relative_list_in_fnc[4][1]

        list_x[4]=relative_list_in_fnc[3][0]
        list_y[4]=relative_list_in_fnc[3][1]

        list_x[3]=relative_list_in_fnc[2][0]
        list_y[3]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[5]=(list_x[6] + list_x[4]) / 2
        list_y[5]=(list_y[6] + list_y[4]) / 2

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list_in_fnc)==6):
        list_x[6]=relative_list_in_fnc[5][0]
        list_y[6]=relative_list_in_fnc[5][1]

        list_x[5]=relative_list_in_fnc[4][0]
        list_y[5]=relative_list_in_fnc[4][1]

        list_x[4]=relative_list_in_fnc[3][0]
        list_y[4]=relative_list_in_fnc[3][1]

        list_x[3]=relative_list_in_fnc[2][0]
        list_y[3]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

        # filling 0 values

        list_x[2]=(list_x[3] + list_x[1]) / 2
        list_y[2]=(list_y[3] + list_y[1]) / 2
    
    if(len(relative_list_in_fnc)==7):
        list_x[6]=relative_list_in_fnc[6][0]
        list_y[6]=relative_list_in_fnc[6][1]
        
        list_x[5]=relative_list_in_fnc[5][0]
        list_y[5]=relative_list_in_fnc[5][1]

        list_x[4]=relative_list_in_fnc[4][0]
        list_y[4]=relative_list_in_fnc[4][1]

        list_x[3]=relative_list_in_fnc[3][0]
        list_y[3]=relative_list_in_fnc[3][1]

        list_x[2]=relative_list_in_fnc[2][0]
        list_y[2]=relative_list_in_fnc[2][1]

        list_x[1]=relative_list_in_fnc[1][0]
        list_y[1]=relative_list_in_fnc[1][1]

        list_x[0]=relative_list_in_fnc[0][0]
        list_y[0]=relative_list_in_fnc[0][1]

    return list_x,list_y

In [2]:
print(relative_list)
count = 1
print("Fill coords:")
b = [3,6]
print(fill_coordinates([ relative_list[i] for i in b]))
# for i in relative_list:
#     print(fill_coordinates(relative_list[-count:]))
#     count = count+1


NameError: name 'relative_list' is not defined